# 🎨 Data Designer Tutorial: Structured Outputs and Jinja Expressions

#### 📚 What you'll learn

In this notebook, we will continue our exploration of Data Designer, demonstrating more advanced data generation using structured outputs and Jinja expressions.

If this is your first time using Data Designer, we recommend starting with the [first notebook](https://nvidia-nemo.github.io/DataDesigner/latest/notebooks/1-the-basics/) in this tutorial series.


### 📦 Import the essentials

- The `essentials` module provides quick access to the most commonly used objects.


In [1]:
from data_designer.essentials import (
    CategorySamplerParams,
    ChatCompletionInferenceParams,
    DataDesigner,
    DataDesignerConfigBuilder,
    ExpressionColumnConfig,
    LLMStructuredColumnConfig,
    ModelConfig,
    PersonFromFakerSamplerParams,
    SamplerColumnConfig,
    SamplerType,
    SubcategorySamplerParams,
)

### ⚙️ Initialize the Data Designer interface

- `DataDesigner` is the main object that is used to interface with the library.

- When initialized without arguments, the [default model providers](https://nvidia-nemo.github.io/DataDesigner/latest/concepts/models/default-model-settings/) are used.


In [2]:
data_designer = DataDesigner()

### 🎛️ Define model configurations

- Each `ModelConfig` defines a model that can be used during the generation process.

- The "model alias" is used to reference the model in the Data Designer config (as we will see below).

- The "model provider" is the external service that hosts the model (see the [model config](https://nvidia-nemo.github.io/DataDesigner/latest/concepts/models/default-model-settings/) docs for more details).

- By default, we use [build.nvidia.com](https://build.nvidia.com/models) as the model provider.


In [3]:
# This name is set in the model provider configuration.
MODEL_PROVIDER = "nvidia"

# The model ID is from build.nvidia.com.
MODEL_ID = "nvidia/nemotron-3-nano-30b-a3b"

# We choose this alias to be descriptive for our use case.
MODEL_ALIAS = "nemotron-nano-v3"

model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=ChatCompletionInferenceParams(
            temperature=1.0,
            top_p=1.0,
            max_tokens=2048,
            extra_body={"chat_template_kwargs": {"enable_thinking": False}},
        ),
    )
]

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- The list of model configs is provided to the builder at initialization.


In [4]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

### 🧑‍🎨 Designing our data

- We will again create a product review dataset, but this time we will use structured outputs and Jinja expressions.

- Structured outputs let you specify the exact schema of the data you want to generate.

- Data Designer supports schemas specified using either json schema or Pydantic data models (recommended).

<br>

We'll define our structured outputs using [Pydantic](https://docs.pydantic.dev/latest/) data models

> 💡 **Why Pydantic?**
>
> - Pydantic models provide better IDE support and type validation.
>
> - They are more Pythonic than raw JSON schemas.
>
> - They integrate seamlessly with Data Designer's structured output system.


In [5]:
from decimal import Decimal
from typing import Literal

from pydantic import BaseModel, Field


# We define a Product schema so that the name, description, and price are generated
# in one go, with the types and constraints specified.
class Product(BaseModel):
    name: str = Field(description="The name of the product")
    description: str = Field(description="A description of the product")
    price: Decimal = Field(description="The price of the product", ge=10, le=1000, decimal_places=2)


class ProductReview(BaseModel):
    rating: int = Field(description="The rating of the product", ge=1, le=5)
    customer_mood: Literal["irritated", "mad", "happy", "neutral", "excited"] = Field(
        description="The mood of the customer"
    )
    review: str = Field(description="A review of the product")

Next, let's design our product review dataset using a few more tricks compared to the previous notebook.


In [6]:
# Since we often only want a few attributes from Person objects, we can
# set drop=True in the column config to drop the column from the final dataset.
config_builder.add_column(
    SamplerColumnConfig(
        name="customer",
        sampler_type=SamplerType.PERSON_FROM_FAKER,
        params=PersonFromFakerSamplerParams(),
        drop=True,
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="product_category",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=[
                "Electronics",
                "Clothing",
                "Home & Kitchen",
                "Books",
                "Home Office",
            ],
        ),
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="product_subcategory",
        sampler_type=SamplerType.SUBCATEGORY,
        params=SubcategorySamplerParams(
            category="product_category",
            values={
                "Electronics": [
                    "Smartphones",
                    "Laptops",
                    "Headphones",
                    "Cameras",
                    "Accessories",
                ],
                "Clothing": [
                    "Men's Clothing",
                    "Women's Clothing",
                    "Winter Coats",
                    "Activewear",
                    "Accessories",
                ],
                "Home & Kitchen": [
                    "Appliances",
                    "Cookware",
                    "Furniture",
                    "Decor",
                    "Organization",
                ],
                "Books": [
                    "Fiction",
                    "Non-Fiction",
                    "Self-Help",
                    "Textbooks",
                    "Classics",
                ],
                "Home Office": [
                    "Desks",
                    "Chairs",
                    "Storage",
                    "Office Supplies",
                    "Lighting",
                ],
            },
        ),
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="target_age_range",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(values=["18-25", "25-35", "35-50", "50-65", "65+"]),
    )
)

# Sampler columns support conditional params, which are used if the condition is met.
# In this example, we set the review style to rambling if the target age range is 18-25.
# Note conditional parameters are only supported for Sampler column types.
config_builder.add_column(
    SamplerColumnConfig(
        name="review_style",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=["rambling", "brief", "detailed", "structured with bullet points"],
            weights=[1, 2, 2, 1],
        ),
        conditional_params={
            "target_age_range == '18-25'": CategorySamplerParams(values=["rambling"]),
        },
    )
)

# Optionally validate that the columns are configured correctly.
data_designer.validate(config_builder)

[22:47:42] [INFO] ✅ Validation passed


Next, we will use more advanced Jinja expressions to create new columns.

Jinja expressions let you:

- Access nested attributes: `{{ customer.first_name }}`

- Combine values: `{{ customer.first_name }} {{ customer.last_name }}`

- Use conditional logic: `{% if condition %}...{% endif %}`


In [7]:
# We can create new columns using Jinja expressions that reference
# existing columns, including attributes of nested objects.
config_builder.add_column(
    ExpressionColumnConfig(name="customer_name", expr="{{ customer.first_name }} {{ customer.last_name }}")
)

config_builder.add_column(ExpressionColumnConfig(name="customer_age", expr="{{ customer.age }}"))

config_builder.add_column(
    LLMStructuredColumnConfig(
        name="product",
        prompt=(
            "Create a product in the '{{ product_category }}' category, focusing on products  "
            "related to '{{ product_subcategory }}'. The target age range of the ideal customer is "
            "{{ target_age_range }} years old. The product should be priced between $10 and $1000."
        ),
        output_format=Product,
        model_alias=MODEL_ALIAS,
    )
)

# We can even use if/else logic in our Jinja expressions to create more complex prompt patterns.
config_builder.add_column(
    LLMStructuredColumnConfig(
        name="customer_review",
        prompt=(
            "Your task is to write a review for the following product:\n\n"
            "Product Name: {{ product.name }}\n"
            "Product Description: {{ product.description }}\n"
            "Price: {{ product.price }}\n\n"
            "Imagine your name is {{ customer_name }} and you are from {{ customer.city }}, {{ customer.state }}. "
            "Write the review in a style that is '{{ review_style }}'."
            "{% if target_age_range == '18-25' %}"
            "Make sure the review is more informal and conversational.\n"
            "{% else %}"
            "Make sure the review is more formal and structured.\n"
            "{% endif %}"
            "The review field should contain only the review, no other text."
        ),
        output_format=ProductReview,
        model_alias=MODEL_ALIAS,
    )
)

data_designer.validate(config_builder)

[22:47:42] [INFO] ✅ Validation passed


### 🔁 Iteration is key – preview the dataset!

1. Use the `preview` method to generate a sample of records quickly.

2. Inspect the results for quality and format issues.

3. Adjust column configurations, prompts, or parameters as needed.

4. Re-run the preview until satisfied.


In [8]:
preview = data_designer.preview(config_builder, num_records=2)

[22:47:42] [INFO] 👀 Preview generation in progress


[22:47:42] [INFO] ✅ Validation passed


[22:47:42] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph


[22:47:42] [INFO] 🩺 Running health checks for models...


[22:47:42] [INFO]   |-- 👀 Checking 'nvidia/nemotron-3-nano-30b-a3b' in provider named 'nvidia' for model alias 'nemotron-nano-v3'...


...

[22:47:45] [INFO]   |-- 🧩 column: 'customer_name'


[22:47:45] [INFO]   |-- 🧩 column: 'customer_age'


[22:47:45] [INFO]   |-- 🗂️ column: 'product'


[22:47:45] [INFO]   |-- 🗂️ column: 'customer_review'


[22:47:45] [INFO] 🎊 Preview complete!


In [9]:
# Run this cell multiple times to cycle through the 2 preview records.
preview.display_sample_record()

                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                ┃ Value                                                                                     ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ product_category    │ Home & Kitchen                                                                            │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ product_subcategory │ Furniture                                                                                 │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ target_age_range    │ 65+                                                                                       │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ review_style        │ rambling                                                                                  │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ customer_name       │ Katie Anderson                                                                            │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ customer_age        │ 69                                                                                        │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ product             │ {                                                                                         │
│                     │     'name': 'ErgoLift Comfort Seat',                                                      │
│                     │     'description': 'A sturdy, slip‑resistant wooden chair with a raised, padded seat and  │
│                     │ armrests, designed to make standing and sitting easier for seniors. Features a supportive │
│                     │ back, rounded edges for safety, and a compact footprint that fits neatly in any kitchen   │
│                     │ or dining area.',                                                                         │
│                     │     'price': 129.99                                                                       │
│                     │ }                                                                                         │
├─────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ customer_review     │ {                                                                                         │
│                     │     'rating': 4,                                                                          │
│                     │     'customer_mood': 'happy',                                                             │
│                     │     'review': 'The ErgoLift Comfort Seat arrived promptly and, after a brief assembly     │
│                     │ that required only a couple of wooden screws, it settled into my kitchen nook with an     │
│                     │ effortless grace. The sturdy, slip‑resistant wooden frame gives the chair a reassuring    │
│                     │ heft, while the raised, padded seat cradles my hips like a gentle embrace, and the        │
│                     │ armrests—soft yet firm—allow me to push up and lower myself without the wobble I once     │
│                     │ feared. Its supportive back arches just enough to encourage upright posture, and the      │
│                     │ rounded edges eliminate any worr

In [10]:
# The preview dataset is available as a pandas DataFrame.
preview.dataset

,product_category,product_subcategory,target_age_range,review_style,customer_name,customer_age,product,customer_review
0,Home & Kitchen,Furniture,65+,rambling,Katie Anderson,69,"{'name': 'ErgoLift Comfort Seat', 'description...","{'rating': 4, 'customer_mood': 'happy', 'revie..."
1,Clothing,Women's Clothing,18-25,rambling,Michael Hernandez,26,"{'name': 'Silk Spindle Ribbon Dress', 'descrip...","{'rating': 5, 'customer_mood': 'happy', 'revie..."


### 📊 Analyze the generated data

- Data Designer automatically generates a basic statistical analysis of the generated data.

- This analysis is available via the `analysis` property of generation result objects.


In [11]:
# Print the analysis as a table.
preview.analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2                               │ 8                               │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                      ┃        data type ┃               number unique values ┃         sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ product_category                 │           string │                         2 (100.0%) │             category │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ product_subcategory              │           string │                         2 (100.0%) │          subcategory │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ target_age_range                 │           string │                         2 (100.0%) │             category │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ review_style                     │           string │                          1 (50.0%) │             category │
└──────────────────────────────────┴──────────────────┴────────────────────────────────────┴──────────────────────┘
                                                                                                                   
                                                                                                                   
                                             🗂️ LLM-Structured Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                       ┃               ┃                            ┃     prompt tokens ┃      completion tokens ┃
┃ column name           ┃     data type ┃       number unique values ┃        per record ┃             per record ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ product               │          dict │                 2 (100.0%) │     265.5 +/- 0.5 │           84.0 +/- 8.5 │
├───────────────────────┼───────────────┼────────────────────────────┼───────────────────┼────────────────────────┤
│ customer_review       │          dict │                 2 (100.0%) │     338.0 +/- 6.0 │         255.0 +/- 48.1 │
└───────────────────────┴───────────────┴────────────────────────────┴───────────────────┴────────────────────────┘
                                                                                                                   
                                                        

### 🆙 Scale up!

- Happy with your preview data?

- Use the `create` method to submit larger Data Designer generation jobs.


In [12]:
results = data_designer.create(config_builder, num_records=10, dataset_name="tutorial-2")

[22:47:46] [INFO] 🎨 Creating Data Designer dataset


[22:47:46] [INFO] ✅ Validation passed


[22:47:46] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph


[22:47:46] [INFO] 🩺 Running health checks for models...


[22:47:46] [INFO]   |-- 👀 Checking 'nvidia/nemotron-3-nano-30b-a3b' in provider named 'nvidia' for model alias 'nemotron-nano-v3'...


...

[22:47:56] [INFO]   |-- 🎲 column: 'review_style'


[22:47:56] [INFO]   |-- 🧩 column: 'customer_name'


[22:47:56] [INFO]   |-- 🧩 column: 'customer_age'


[22:47:56] [INFO]   |-- 🗂️ column: 'product'


[22:47:56] [INFO]   |-- 🗂️ column: 'customer_review'


In [13]:
# Load the generated dataset as a pandas DataFrame.
dataset = results.load_dataset()

dataset.head()

,product_category,product_subcategory,target_age_range,review_style,customer_name,customer_age,product,customer_review
0,Books,Fiction,18-25,rambling,Heather Miller,105,{'description': 'A beautifully designed sketch...,"{'customer_mood': 'excited', 'rating': 5, 'rev..."
1,Electronics,Smartphones,25-35,detailed,Emily Whitney,63,"{'description': 'A sleek, feature‑rich smartwa...","{'customer_mood': 'excited', 'rating': 5, 'rev..."
2,Electronics,Laptops,35-50,rambling,Victoria Cooper,57,{'description': 'A compact and lightweight USB...,"{'customer_mood': 'excited', 'rating': 5, 'rev..."
3,Books,Textbooks,65+,brief,Kevin Carroll,28,"{'description': 'A comprehensive, large-print ...","{'customer_mood': 'happy', 'rating': 4, 'revie..."
4,Clothing,Accessories,50-65,rambling,Crystal Soto,86,{'description': 'A classy collection of three ...,"{'customer_mood': 'happy', 'rating': 5, 'revie..."


In [14]:
# Load the analysis results into memory.
analysis = results.load_analysis()

analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 10                              │ 8                               │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                      ┃        data type ┃               number unique values ┃         sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ product_category                 │           string │                          5 (50.0%) │             category │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ product_subcategory              │           string │                          9 (90.0%) │          subcategory │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ target_age_range                 │           string │                          5 (50.0%) │             category │
├──────────────────────────────────┼──────────────────┼────────────────────────────────────┼──────────────────────┤
│ review_style                     │           string │                          4 (40.0%) │             category │
└──────────────────────────────────┴──────────────────┴────────────────────────────────────┴──────────────────────┘
                                                                                                                   
                                                                                                                   
                                             🗂️ LLM-Structured Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓
┃                      ┃               ┃                            ┃       prompt tokens ┃     completion tokens ┃
┃ column name          ┃     data type ┃       number unique values ┃          per record ┃            per record ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩
│ product              │          dict │                10 (100.0%) │       265.0 +/- 0.8 │         97.0 +/- 27.7 │
├──────────────────────┼───────────────┼────────────────────────────┼─────────────────────┼───────────────────────┤
│ customer_review      │          dict │                10 (100.0%) │      351.5 +/- 25.8 │       219.5 +/- 234.9 │
└──────────────────────┴───────────────┴────────────────────────────┴─────────────────────┴───────────────────────┘
                                                                                                                   
                                                        

## ⏭️ Next Steps

Check out the following notebook to learn more about:

- [Seeding synthetic data generation with an external dataset](https://nvidia-nemo.github.io/DataDesigner/latest/notebooks/3-seeding-with-a-dataset/)

- [Providing images as context](https://nvidia-nemo.github.io/DataDesigner/latest/notebooks/4-providing-images-as-context/)A
